In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import string
import re
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.under_sampling import TomekLinks
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

%matplotlib inline

In [5]:
customer=pd.read_csv('data\\Customer_Data.csv')
customer.head()

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Business Partner,Customer No.,Partner Type,Data Origin,Title,Marital Status,Occupation,Date of Birth,Death date
0,31,1,1.0,Z001,NaN,NaN,NaN,NaN,NaN
1,32,2,1.0,Z005,NaN,NaN,NaN,NaN,NaN
2,34,4,1.0,Z005,NaN,2.0,NaN,NaN,NaN
3,35,5,1.0,Z005,NaN,NaN,NaN,NaN,NaN
4,42,7,1.0,Z008,NaN,NaN,NaN,NaN,NaN


In [6]:
invoice=pd.read_csv('data\\Final_invoice.csv')
invoice.head()

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (9,10,14,15,21,41,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Amt Rcvd From Custom,Amt Rcvd From Ins Co,Area / Locality,CGST(14%),CGST(2.5%),CGST(6%),CGST(9%),CITY,Cash /Cashless Type,...,Service Advisor Name,TDS amount,Technician Name,Total Amt Wtd Tax.,Total CGST,Total GST,Total IGST,Total SGST/UGST,Total Value,User ID
0,0,0.0,0.0,MAJIWADA,0.0,0.0,0.0,0.0,Thane,NaN,...,NaN,0.0,RUPESH,4051.95,0.0,0.0,0.0,0.0,4051.95,BC01FS1
1,1,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,...,NaN,0.0,PRASHANT,1001.29,0.0,0.0,0.0,0.0,1001.29,BC01SA2
2,2,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE[W],NaN,...,NaN,0.0,IMRAN,233.14,0.0,0.0,0.0,0.0,233.14,BC01SA2
3,3,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,...,NaN,0.0,PRASHANT,0.00,0.0,0.0,0.0,0.0,0.00,BC01SA2
4,4,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE,NaN,...,NaN,0.0,DAYANAND,2747.12,0.0,0.0,0.0,0.0,2747.12,BC01SA2


In [7]:
JTD=pd.read_csv('data\\JTD.csv')
JTD.head()

,Unnamed: 0,DBM Order,Order Item,Material,Labor Value Number,Description,Item Category,Order Quantity,Target quantity UoM,Net value
0,0,22385,180,OSLCC_SRV31,NaN,MACHINING RR.SUSPENSION CHARGE (MAJOR),P010,1.2,MIN,1869.87
1,1,23910,10,1DM279018170117,NaN,ASSY.OIL COOLER (6-PLATE) (VOR,P002,1.0,EA,2151.13
2,2,23910,40,1IS150203BE01,NaN,FRONT WHEEL BEARING/ LUBRICATE,P002,1.0,EA,2552.92
3,3,23910,60,1DM279001990173,NaN,ASSY.CYL.HEAD W /O INJ.(NA W /O,P002,1.0,EA,20360.14
4,4,23910,70,1DM88552790S001SA,NaN,Engine Overhaul Kit -Indica Di,P002,1.0,EA,10691.63


In [12]:
plant_master=pd.read_csv('data\\Plant Master.csv')
plant_master

,Plant,Name 1,Valuation Area,Customer no. - plant,Vendor number plant,Factory calendar,Name 2,House number and street,PO Box,Postal Code,City,Sales organization,State
0,BC07,NERKUNDRAM,BC07,PLBC07,PLBC07,IN,NaN,"No.117 Poonamallee High Road,",600107.0,600107,Chennai,MFCS,Tamil Nadu
1,BC30,SAS NAGAR - MOHALI,BC30,PLBC30,PLBC30,IN,NaN,"C-118, Phase 07,",160055.0,160055,Mohali,MFCS,Punjab
2,X001,X001 - AUTO PLANET WHEELS,X001,PLX001,NaN,IN,Thane,"Unit No.3, Kothari Compound,",400610.0,400610,Thane,MFCA,Maharashtra
3,X002,X002 - BHANDARI INDUSTRIES PVT,X002,PLX002,NaN,IN,Ghaziabad,"18/1, C Site 4, Sahibabad",201010.0,201010,Ghaziabad,MFCA,Uttar Pradesh
4,X003,X003 - Om Sri Satya Sai Automo,X003,PLX003,NaN,IN,Bengaluru,"#75, Sy.No. 83/2, Panathur Mai",560087.0,560087,Bengaluru,MFCB,Karnataka
5,X005,X005 - KLN MOTORS,X005,PLX005,NaN,IN,Chennai,"No.8 (NP) Developed Plot,",600032.0,600032,Chennai,MFCB,Tamil Nadu
6,X006,X006 - A.M.MOTORS,X006,PLX006,NaN,IN,Mumbai,"8, Mira, Behind Ajit Palace Ho",401104.0,401104,Thane,MFCB,Maharashtra
7,X007,X007 - OM SRI SATYA SAI AUTOMO,X007,PLX007,NaN,IN,Bengaluru,"Seegehalli main road,",560037.0,560037,Bengaluru,MFCB,Karnataka
8,X008,X008- AAKASH AUTO (Puducherry),X008,PLX008,NaN,IN,Puducherry,"145/2, Villiyanur main road,",605110.0,605110,Puducherry,MFCC,Puducherry
9,X010,X010 - LVM MOTORS,X010,PLX010,NaN,IN,Chennai,"B- 49 Emerald Nagar, SIPCOT,",632406.0,632406,Ranipet,MFCD,Tamil Nadu


In [9]:
JTD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5619484 entries, 0 to 5619483
Data columns (total 10 columns):
Unnamed: 0             int64
DBM Order              int64
Order Item             int64
Material               object
Labor Value Number     object
Description            object
Item Category          object
Order Quantity         float64
Target quantity UoM    object
Net value              float64
dtypes: float64(2), int64(3), object(5)
memory usage: 428.7+ MB


In [10]:
invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492314 entries, 0 to 492313
Data columns (total 59 columns):
Unnamed: 0              492314 non-null int64
Amt Rcvd From Custom    295028 non-null float64
Amt Rcvd From Ins Co    295028 non-null float64
Area / Locality         468424 non-null object
CGST(14%)               295028 non-null float64
CGST(2.5%)              295028 non-null float64
CGST(6%)                295028 non-null float64
CGST(9%)                295028 non-null float64
CITY                    492313 non-null object
Cash /Cashless Type     27420 non-null object
Claim No.               4804 non-null object
Cust Type               492314 non-null object
Customer No.            492314 non-null object
District                492314 non-null object
Expiry Date             22231 non-null object
Gate Pass Date          40915 non-null object
Gate Pass Time          492314 non-null object
IGST(12%)               295028 non-null float64
IGST(18%)               295028 non-null fl

In [17]:
invoice['Service Advisor Name'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Service Advisor Name, dtype: float64

In [14]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555338 entries, 0 to 555337
Data columns (total 9 columns):
Business Partner    555338 non-null object
Customer No.        555338 non-null object
Partner Type        555260 non-null float64
Data Origin         555202 non-null object
Title               509031 non-null float64
Marital Status      16037 non-null float64
Occupation          14558 non-null float64
Date of Birth       13444 non-null object
Death date          663 non-null object
dtypes: float64(4), object(5)
memory usage: 38.1+ MB


In [29]:
customer_invoice = pd.DataFrame(pd.merge(customer, invoice, left_on='Customer No.', right_on='Customer No.',how='left'))
customer_invoice.head()

,Business Partner,Customer No.,Partner Type,Data Origin,Title,Marital Status,Occupation,Date of Birth,Death date,Unnamed: 0,...,Service Advisor Name,TDS amount,Technician Name,Total Amt Wtd Tax.,Total CGST,Total GST,Total IGST,Total SGST/UGST,Total Value,User ID
0,31,1,1.0,Z001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32,2,1.0,Z005,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34,4,1.0,Z005,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35,5,1.0,Z005,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42,7,1.0,Z008,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
customer_invoice_JTD=pd.DataFrame(pd.merge(customer_invoice, JTD, left_on='Job Card No', right_on='DBM Order',how='left'))
customer_invoice_JTD.head()

,Business Partner,Customer No.,Partner Type,Data Origin,Title,Marital Status,Occupation,Date of Birth,Death date,Unnamed: 0_x,...,Unnamed: 0_y,DBM Order,Order Item,Material,Labor Value Number,Description,Item Category,Order Quantity,Target quantity UoM,Net value
0,31,1,1.0,Z001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32,2,1.0,Z005,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34,4,1.0,Z005,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35,5,1.0,Z005,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42,7,1.0,Z008,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
customer_invoice.shape

(571416, 67)

In [27]:
customer.shape

(555338, 9)

In [28]:
invoice.shape

(492314, 59)

In [31]:
JTD.shape

(5619484, 10)

In [43]:
customer_invoice_JTD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705757 entries, 0 to 705756
Data columns (total 77 columns):
Business Partner        705757 non-null object
Customer No.            705757 non-null object
Partner Type            692134 non-null float64
Data Origin             676542 non-null object
Title                   527870 non-null float64
Marital Status          16037 non-null float64
Occupation              14558 non-null float64
Date of Birth           13444 non-null object
Death date              663 non-null object
Unnamed: 0_x            154068 non-null float64
Amt Rcvd From Custom    144607 non-null float64
Amt Rcvd From Ins Co    144607 non-null float64
Area / Locality         3269 non-null object
CGST(14%)               144607 non-null float64
CGST(2.5%)              144607 non-null float64
CGST(6%)                144607 non-null float64
CGST(9%)                144607 non-null float64
CITY                    154068 non-null object
Cash /Cashless Type     7484 non-null ob

In [44]:
customer_invoice_JTD_plant=pd.DataFrame(pd.merge(customer_invoice_JTD, plant_master, left_on='Plant', right_on='Plant',how='left'))
customer_invoice_JTD_plant.head()

,Business Partner,Customer No.,Partner Type,Data Origin,Title,Marital Status,Occupation,Date of Birth,Death date,Unnamed: 0_x,...,Customer no. - plant,Vendor number plant,Factory calendar,Name 2,House number and street,PO Box,Postal Code,City,Sales organization,State
0,31,1,1.0,Z001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32,2,1.0,Z005,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34,4,1.0,Z005,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35,5,1.0,Z005,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42,7,1.0,Z008,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
customer_invoice_JTD_plant.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705757 entries, 0 to 705756
Data columns (total 89 columns):
Business Partner           705757 non-null object
Customer No.               705757 non-null object
Partner Type               692134 non-null float64
Data Origin                676542 non-null object
Title                      527870 non-null float64
Marital Status             16037 non-null float64
Occupation                 14558 non-null float64
Date of Birth              13444 non-null object
Death date                 663 non-null object
Unnamed: 0_x               154068 non-null float64
Amt Rcvd From Custom       144607 non-null float64
Amt Rcvd From Ins Co       144607 non-null float64
Area / Locality            3269 non-null object
CGST(14%)                  144607 non-null float64
CGST(2.5%)                 144607 non-null float64
CGST(6%)                   144607 non-null float64
CGST(9%)                   144607 non-null float64
CITY                       154068 no

In [48]:
final_data=customer_invoice_JTD_plant.copy()

### Check missing values

In [47]:
# Function to calculate missing values by column
def missing_values_table(df):
        "Function which will return null values percentage by column"
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [50]:
missing_values_table(customer)

Your selected dataframe has 9 columns.
There are 7 columns that have missing values.


,Missing Values,% of Total Values
Death date,554675,99.9
Date of Birth,541894,97.6
Occupation,540780,97.4
Marital Status,539301,97.1
Title,46307,8.3
Data Origin,136,0.0
Partner Type,78,0.0


In [55]:
customer_n=customer.drop(columns=['Death date', 'Date of Birth','Occupation','Marital Status'])
customer_n.head()

,Business Partner,Customer No.,Partner Type,Data Origin,Title
0,31,1,1.0,Z001,NaN
1,32,2,1.0,Z005,NaN
2,34,4,1.0,Z005,NaN
3,35,5,1.0,Z005,NaN
4,42,7,1.0,Z008,NaN


In [177]:
missing_values_table(invoice_n)

Your selected dataframe has 34 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [59]:
invoice_n=invoice.drop(columns=['Service Advisor Name', 'Claim No.','Policy no.','Expiry Date','Insurance Company'])
invoice_n.head()

,Unnamed: 0,Amt Rcvd From Custom,Amt Rcvd From Ins Co,Area / Locality,CGST(14%),CGST(2.5%),CGST(6%),CGST(9%),CITY,Cash /Cashless Type,...,SGST/UGST(9%),TDS amount,Technician Name,Total Amt Wtd Tax.,Total CGST,Total GST,Total IGST,Total SGST/UGST,Total Value,User ID
0,0,0.0,0.0,MAJIWADA,0.0,0.0,0.0,0.0,Thane,NaN,...,0.0,0.0,RUPESH,4051.95,0.0,0.0,0.0,0.0,4051.95,BC01FS1
1,1,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,...,0.0,0.0,PRASHANT,1001.29,0.0,0.0,0.0,0.0,1001.29,BC01SA2
2,2,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE[W],NaN,...,0.0,0.0,IMRAN,233.14,0.0,0.0,0.0,0.0,233.14,BC01SA2
3,3,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,...,0.0,0.0,PRASHANT,0.00,0.0,0.0,0.0,0.0,0.00,BC01SA2
4,4,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE,NaN,...,0.0,0.0,DAYANAND,2747.12,0.0,0.0,0.0,0.0,2747.12,BC01SA2


In [60]:
missing_values_table(JTD)

Your selected dataframe has 10 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
Labor Value Number,3774033,67.2
Material,51629,0.9
Description,51629,0.9
Target quantity UoM,51629,0.9


In [ ]:
JTD['Labor Value Number'].value_counts()

### Customer Preprocessing

In [176]:
customer_n.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555338 entries, 0 to 555337
Data columns (total 5 columns):
Business Partner    555338 non-null object
Customer No.        555338 non-null object
Partner Type        555338 non-null float64
Data Origin         555338 non-null object
Title               555338 non-null float64
dtypes: float64(2), object(3)
memory usage: 21.2+ MB


In [68]:
customer_n['Data Origin'].value_counts()

Z005    149861
Z006    121660
Z001     63237
Z002     38326
Z004     36801
Z019     29780
Z021     28828
Z018     15382
Z013     13967
Z008     12122
Z016      8118
Z011      8023
Z010      6260
Z017      5685
Z003      5082
Z014      4014
Z012      3840
Z007      2069
Z015       745
Z009       735
Z020       590
Z022        77
Name: Data Origin, dtype: int64

In [65]:
customer_n['Title']=customer_n['Title'].fillna(5)
customer_n['Title'].value_counts()

2.0    483932
5.0     46307
1.0     25099
Name: Title, dtype: int64

In [69]:
customer_n.head()

,Business Partner,Customer No.,Partner Type,Data Origin,Title
0,31,1,1.0,Z001,5.0
1,32,2,1.0,Z005,5.0
2,34,4,1.0,Z005,5.0
3,35,5,1.0,Z005,5.0
4,42,7,1.0,Z008,5.0


In [77]:
customer_n.groupby('Data Origin')['Partner Type'].count().sort_values()

Data Origin
Z022        77
Z020       590
Z009       735
Z015       745
Z007      2069
Z012      3840
Z014      4014
Z003      5082
Z017      5685
Z010      6260
Z011      8023
Z016      8118
Z008     12122
Z013     13967
Z018     15382
Z021     28828
Z019     29780
Z004     36801
Z002     38326
Z001     63237
Z006    121660
Z005    149861
Name: Partner Type, dtype: int64

In [78]:
customer_n['Data Origin']=customer_n['Data Origin'].fillna('Z005')
customer_n['Data Origin'].value_counts()

Z005    149997
Z006    121660
Z001     63237
Z002     38326
Z004     36801
Z019     29780
Z021     28828
Z018     15382
Z013     13967
Z008     12122
Z016      8118
Z011      8023
Z010      6260
Z017      5685
Z003      5082
Z014      4014
Z012      3840
Z007      2069
Z015       745
Z009       735
Z020       590
Z022        77
Name: Data Origin, dtype: int64

### Invoice Processing

In [171]:
invoice_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490789 entries, 0 to 492313
Data columns (total 35 columns):
Unnamed: 0              490789 non-null int64
Amt Rcvd From Custom    490789 non-null float64
Amt Rcvd From Ins Co    490789 non-null float64
Area / Locality         490789 non-null float64
CITY                    490789 non-null object
Cash /Cashless Type     490789 non-null object
Cust Type               490789 non-null object
Customer No.            490789 non-null object
District                490789 non-null object
Gate Pass Date          490789 non-null object
Gate Pass Time          490789 non-null object
Invoice Date            490789 non-null object
Invoice No              490789 non-null int64
Invoice Time            490789 non-null object
Job Card No             490789 non-null int64
JobCard Date            490789 non-null object
JobCard Time            490789 non-null object
KMs Reading             490789 non-null int64
Labour Total            490789 non-null floa

In [131]:
invoice_n['Amt Rcvd From Custom']=invoice_n['Amt Rcvd From Custom'].fillna(0)
invoice_n['Amt Rcvd From Ins Co']=invoice_n['Amt Rcvd From Ins Co'].fillna(0)

In [170]:
invoice_n['Technician Name']=invoice_n['Technician Name'].fillna('Unknown')

In [169]:
invoice_n['Technician Name'].value_counts()

rajesh                 2122
RAJESH                 1976
SAYYAD                 1906
suresh                 1715
MAHESH                 1652
MANOJ                  1648
sayyad                 1482
saravanan              1479
ramesh                 1398
IMRAN                  1387
mahesh                 1365
SARAVANAN              1309
amit                   1304
murugan                1287
pankaj                 1276
ARUN                   1157
JK                     1145
HARSHAD                1104
praveen                1099
satish                 1098
gurjeet                1085
SURESH                 1079
ANIL                   1073
ali                    1052
SATISH                 1032
arun                   1023
KAMAL                   979
anjani                  978
SAMEER                  969
imran                   963
                       ... 
JASPAL WASHING BOY        1
KUMAR/RAJAN               1
GANESH/MAHESH             1
MANOJ /JAWED              1
DAAVARAJI.S         

In [139]:
invoice_n.drop(columns=['CGST(14%)','CGST(2.5%)','CGST(6%)','CGST(9%)','IGST(12%)','IGST(18%)','IGST(28%)','IGST(5%)'],inplace=True)

In [160]:
invoice_n.drop(columns=['SGST/UGST(14%)','SGST/UGST(2.5%)','SGST/UGST(6%)','SGST/UGST(9%)','Total CGST','Total GST','Total IGST','Total SGST/UGST'],inplace=True)

In [167]:
invoice_n.drop(columns=['ODN No.','Outstanding Amt','TDS amount'],inplace=True)

In [175]:
invoice_n.drop(columns=['Total Value'],inplace=True)

In [142]:
invoice_n['Cash /Cashless Type']=invoice_n['Cash /Cashless Type'].fillna('Unknown')

In [149]:
inv=invoice_n[['Gate Pass Date','JobCard Date']].copy()

In [155]:
invoice_n['CITY']=invoice_n['CITY'].fillna(invoice_n['Area / Locality'])

In [173]:
invoice_n = invoice_n[pd.notnull(invoice_n['Regn No'])]

In [188]:
missing_values_table(JTD_n)

Your selected dataframe has 10 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [185]:
JTD['Target quantity UoM'].value_counts()

EA     2745660
MIN    2317562
L       504597
GRO         36
Name: Target quantity UoM, dtype: int64

In [181]:
JTD['Labor Value Number']=JTD['Labor Value Number'].fillna('Unknown')

In [187]:
JTD_n = JTD[pd.notnull(JTD['Material'])]

In [189]:
missing_values_table(plant_master)

Your selected dataframe has 13 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
Vendor number plant,436,99.5
Sales organization,5,1.1
Name 2,2,0.5
House number and street,2,0.5
PO Box,1,0.2


In [192]:
plant_master_n=plant_master.drop(columns=['Vendor number plant','Name 2','PO Box'],inplace=True).copy()

KeyError: "['Vendor number plant' 'Name 2' 'PO Box'] not found in axis"

In [198]:
plant_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 10 columns):
Plant                      438 non-null object
Name 1                     438 non-null object
Valuation Area             438 non-null object
Customer no. - plant       438 non-null object
Factory calendar           438 non-null object
House number and street    438 non-null object
Postal Code                438 non-null int64
City                       438 non-null object
Sales organization         438 non-null object
State                      438 non-null object
dtypes: int64(1), object(9)
memory usage: 34.3+ KB


In [197]:
plant_master['Sales organization']=plant_master['Sales organization'].fillna('Unknown')

In [199]:
customer_n.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555338 entries, 0 to 555337
Data columns (total 5 columns):
Business Partner    555338 non-null object
Customer No.        555338 non-null object
Partner Type        555338 non-null float64
Data Origin         555338 non-null object
Title               555338 non-null float64
dtypes: float64(2), object(3)
memory usage: 21.2+ MB


In [204]:
plant_master.to_csv("data\\plant_master.csv"); 

In [210]:
customer_invoice_join= pd.DataFrame(pd.merge(customer_n, invoice_n, left_on='Customer No.', right_on='Customer No.',how='inner'))
customer_invoice_join.head()


,Business Partner,Customer No.,Partner Type,Data Origin,Title,Unnamed: 0,Amt Rcvd From Custom,Amt Rcvd From Ins Co,Area / Locality,CITY,...,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Regn No,Technician Name,Total Amt Wtd Tax.,User ID
0,CS000018,CS000018,1.0,Z005,5.0,787293,0.0,0.0,0.0,Chengalpattu,...,0.00,603204,BC07,NERKUNDRAM,YES,0.0,TN19Q2153,Unknown,49.75,BC07SA2
1,E10007,E10007,1.0,Z001,5.0,693968,0.0,0.0,0.0,Hyderabad,...,573.80,500037,BC04,BALANAGAR,NO,0.0,AP28BF7209,chary,1083.77,BC04FS1
2,E10013,E10013,1.0,Z001,5.0,725171,0.0,0.0,0.0,Hyderabad,...,0.00,500012,BC04,BALANAGAR,NO,0.0,AP28AS8746,suresh,1001.29,BC04FS1
3,E10013,E10013,1.0,Z001,5.0,725362,0.0,0.0,0.0,Hyderabad,...,0.00,500012,BC04,BALANAGAR,NO,0.0,AP28AS8746,suresh,0.00,BC04FS1
4,E10014,E10014,1.0,Z001,2.0,684230,0.0,0.0,0.0,Hyderabad,...,80.34,500037,BC05,MADEENAGUDA,NO,0.0,AP28AT3891,Unknown,785.21,BC05SA1


In [211]:
customer_invoice_join.shape

(19703, 38)

In [209]:
missing_values_table(customer_invoice_join)

Your selected dataframe has 38 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
Business Partner,471071,96.0
Partner Type,471071,96.0
Data Origin,471071,96.0
Title,471071,96.0


In [ ]:
customer_invoice_JTD_join=pd.DataFrame(pd.merge(customer_invoice_join, JTD_n, left_on='Job Card No', right_on='DBM Order',how='left'))
customer_invoice_JTD_join.head()


In [ ]:
customer_invoice_JTD_plant_join=pd.DataFrame(pd.merge(customer_invoice_JTD_join, plant_master, left_on='Plant', right_on='Plant',how='left'))
customer_invoice_JTD_plant_join.head()

In [215]:
JTD_invoice_join=pd.DataFrame(pd.merge(JTD_n,invoice_n, left_on='DBM Order', right_on='Job Card No',how='inner'))
JTD_invoice_join.head()

,Unnamed: 0_x,DBM Order,Order Item,Material,Labor Value Number,Description,Item Category,Order Quantity,Target quantity UoM,Net value,...,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Regn No,Technician Name,Total Amt Wtd Tax.,User ID
0,0,22385,180,OSLCC_SRV31,Unknown,MACHINING RR.SUSPENSION CHARGE (MAJOR),P010,1.2,MIN,1869.87,...,0.00,500035,BC04,BALANAGAR,NO,0.0,AP28AY8040,laxminarayana,1869.87,BC04SA1
1,1,23910,10,1DM279018170117,Unknown,ASSY.OIL COOLER (6-PLATE) (VOR,P002,1.0,EA,2151.13,...,41990.54,411038,BC02,PUNE,NO,0.0,MH12CD3403,SAJID,52351.48,BC02SA1
2,2,23910,40,1IS150203BE01,Unknown,FRONT WHEEL BEARING/ LUBRICATE,P002,1.0,EA,2552.92,...,41990.54,411038,BC02,PUNE,NO,0.0,MH12CD3403,SAJID,52351.48,BC02SA1
3,3,23910,60,1DM279001990173,Unknown,ASSY.CYL.HEAD W /O INJ.(NA W /O,P002,1.0,EA,20360.14,...,41990.54,411038,BC02,PUNE,NO,0.0,MH12CD3403,SAJID,52351.48,BC02SA1
4,4,23910,70,1DM88552790S001SA,Unknown,Engine Overhaul Kit -Indica Di,P002,1.0,EA,10691.63,...,41990.54,411038,BC02,PUNE,NO,0.0,MH12CD3403,SAJID,52351.48,BC02SA1


In [216]:
JTD_invoice_join.shape

(3058847, 44)

In [217]:
missing_values_table(JTD_invoice_join)

Your selected dataframe has 44 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [272]:
JTD_invoice_customer_join=pd.DataFrame(pd.merge(JTD_invoice_join,customer_n, left_on='Customer No.', right_on='Customer No.',how='left'))
JTD_invoice_customer_join.head()

,Unnamed: 0_x,DBM Order,Order Item,Material,Labor Value Number,Description,Item Category,Order Quantity,Target quantity UoM,Net value,...,Print Status,Recovrbl Exp,Regn No,Technician Name,Total Amt Wtd Tax.,User ID,Business Partner,Partner Type,Data Origin,Title
0,0,22385,180,OSLCC_SRV31,Unknown,MACHINING RR.SUSPENSION CHARGE (MAJOR),P010,1.2,MIN,1869.87,...,NO,0.0,AP28AY8040,laxminarayana,1869.87,BC04SA1,NaN,NaN,NaN,NaN
1,1,23910,10,1DM279018170117,Unknown,ASSY.OIL COOLER (6-PLATE) (VOR,P002,1.0,EA,2151.13,...,NO,0.0,MH12CD3403,SAJID,52351.48,BC02SA1,E26912,1.0,Z001,5.0
2,2,23910,40,1IS150203BE01,Unknown,FRONT WHEEL BEARING/ LUBRICATE,P002,1.0,EA,2552.92,...,NO,0.0,MH12CD3403,SAJID,52351.48,BC02SA1,E26912,1.0,Z001,5.0
3,3,23910,60,1DM279001990173,Unknown,ASSY.CYL.HEAD W /O INJ.(NA W /O,P002,1.0,EA,20360.14,...,NO,0.0,MH12CD3403,SAJID,52351.48,BC02SA1,E26912,1.0,Z001,5.0
4,4,23910,70,1DM88552790S001SA,Unknown,Engine Overhaul Kit -Indica Di,P002,1.0,EA,10691.63,...,NO,0.0,MH12CD3403,SAJID,52351.48,BC02SA1,E26912,1.0,Z001,5.0


In [273]:
JTD_invoice_customer_join.shape

(3058847, 48)

In [278]:
missing_values_table(JTD_invoice_customer_join)

Your selected dataframe has 48 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [277]:
JTD_invoice_customer_join['Business Partner']=JTD_invoice_customer_join['Business Partner'].fillna(JTD_invoice_customer_join['Customer No.'])
JTD_invoice_customer_join['Partner Type']=JTD_invoice_customer_join['Partner Type'].fillna(6)
JTD_invoice_customer_join['Data Origin']=JTD_invoice_customer_join['Data Origin'].fillna('NoCustomer')
JTD_invoice_customer_join['Title']=JTD_invoice_customer_join['Title'].fillna(6)

In [275]:
customer_n

,Business Partner,Customer No.,Partner Type,Data Origin,Title
0,31,1,1.0,Z001,5.0
1,32,2,1.0,Z005,5.0
2,34,4,1.0,Z005,5.0
3,35,5,1.0,Z005,5.0
4,42,7,1.0,Z008,5.0
5,45,9,1.0,Z003,5.0
6,46,10,1.0,Z005,5.0
7,47,11,1.0,Z005,5.0
8,49,13,9001.0,Z003,5.0
9,50,14,1.0,Z005,5.0
